<a href="https://colab.research.google.com/github/sankardevisharath/amex-default-prediction/blob/master/notebooks/LightGBM_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%mkdir data
%cd data
%mkdir raw
%cd raw

/content/data
/content/data/raw


In [2]:
!cp /content/drive/MyDrive/amex-default-prediction/data/raw/data_2017-03.parquet /content/data/raw/

In [3]:
!unzip /content/drive/MyDrive/amex-default-prediction/data/amex-default-prediction.zip train_labels.csv

Archive:  /content/drive/MyDrive/amex-default-prediction/data/amex-default-prediction.zip
  inflating: train_labels.csv        


In [4]:
import numpy as np
import pandas as pd

In [5]:
TRAIN_DATA_PATH = '/content/data/raw/data_2017-03.parquet'
TRAIN_LABELS_PATH = '/content/data/raw/train_labels.csv'

In [6]:
df = pd.read_parquet(TRAIN_DATA_PATH)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395630 entries, 0 to 395629
Columns: 190 entries, customer_ID to D_145
dtypes: datetime64[ns](1), float64(185), int64(1), object(3)
memory usage: 573.5+ MB


In [8]:
df['customer_ID'].nunique()

395630

In [9]:
def read_cols(cols):
  df = pd.read_csv(TRAIN_DATA_PATH, usecols=cols)
  df = append_label(df)
  return df

def append_label(source_df):
  return pd.merge(left=source_df, right=train_labels, how='inner')

In [10]:
train_labels = pd.read_csv(TRAIN_LABELS_PATH)

In [11]:
df = append_label(df)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Columns: 191 entries, customer_ID to target
dtypes: datetime64[ns](1), float64(185), int64(2), object(3)
memory usage: 579.5+ MB


In [13]:
cat_cols = ['D_63', 'D_64','B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68']

In [14]:
df[cat_cols].isnull().sum() 

D_63          0
D_64      24578
B_30        199
B_38        199
D_114     14105
D_116     14105
D_117     14105
D_120     14105
D_126     14105
D_66     352773
D_68      24578
dtype: int64

In [15]:
df[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   D_63    395630 non-null  object 
 1   D_64    371052 non-null  object 
 2   B_30    395431 non-null  float64
 3   B_38    395431 non-null  float64
 4   D_114   381525 non-null  float64
 5   D_116   381525 non-null  float64
 6   D_117   381525 non-null  float64
 7   D_120   381525 non-null  float64
 8   D_126   381525 non-null  float64
 9   D_66    42857 non-null   float64
 10  D_68    371052 non-null  float64
dtypes: float64(9), object(2)
memory usage: 36.2+ MB


In [16]:
from sklearn.preprocessing import OrdinalEncoder

In [17]:
 enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [18]:
df[['D_63', 'D_64']]=enc.fit_transform(df[['D_63', 'D_64']])

In [19]:
enc.categories_

[array(['CL', 'CO', 'CR', 'XL', 'XM', 'XZ'], dtype=object),
 array(['-1', 'O', 'R', 'U', None], dtype=object)]

In [20]:
df[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   D_63    395630 non-null  float64
 1   D_64    395630 non-null  float64
 2   B_30    395431 non-null  float64
 3   B_38    395431 non-null  float64
 4   D_114   381525 non-null  float64
 5   D_116   381525 non-null  float64
 6   D_117   381525 non-null  float64
 7   D_120   381525 non-null  float64
 8   D_126   381525 non-null  float64
 9   D_66    42857 non-null   float64
 10  D_68    371052 non-null  float64
dtypes: float64(11)
memory usage: 36.2 MB


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Columns: 191 entries, customer_ID to target
dtypes: datetime64[ns](1), float64(187), int64(2), object(1)
memory usage: 579.5+ MB


In [22]:
df.fillna(0, inplace=True)

In [23]:
df[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   D_63    395630 non-null  float64
 1   D_64    395630 non-null  float64
 2   B_30    395630 non-null  float64
 3   B_38    395630 non-null  float64
 4   D_114   395630 non-null  float64
 5   D_116   395630 non-null  float64
 6   D_117   395630 non-null  float64
 7   D_120   395630 non-null  float64
 8   D_126   395630 non-null  float64
 9   D_66    395630 non-null  float64
 10  D_68    395630 non-null  float64
dtypes: float64(11)
memory usage: 36.2 MB


In [24]:
from sklearn.model_selection import train_test_split


In [25]:
X = df.loc[:, ~df.columns.isin(['customer_ID', 'S_2', 'target'])]
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Columns: 188 entries, P_2 to D_145
dtypes: float64(187), int64(1)
memory usage: 570.5 MB


In [26]:
Y = df.loc[:, df.columns == 'target']
Y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   target  395630 non-null  int64
dtypes: int64(1)
memory usage: 6.0 MB


In [27]:
 X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [28]:
X_train.shape

(316504, 188)

In [29]:
X_test.shape

(79126, 188)

In [30]:
import lightgbm as gbm

In [32]:
gbc = gbm.LGBMClassifier()

In [33]:
gbc.fit(X_train, y_train.values.ravel())

LGBMClassifier()

In [34]:
y_pred = gbc.predict(X_test)

In [35]:
gbc.score(X_test, y_test)

0.8640522710613452

In [36]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [37]:
f1_score(y_test, y_pred)

0.7033615530981993